## **[LeetCode Link](https://leetcode-cn.com/problems/cheapest-flights-within-k-stops/solution/k-zhan-zhong-zhuan-nei-zui-bian-yi-de-hang-ban-b-2/)**

## Bellman-Ford 算法（动态规划）**[LeetCode Link](https://leetcode-cn.com/problems/cheapest-flights-within-k-stops/solution/k-zhan-zhong-zhuan-nei-zui-bian-yi-de-hang-ban-b-2/718368)**
动态规划解法的 dp[i][k] 表示从 src 到节点 i，在最多换乘 k 站的情况下，最小的费用。

初始值是 dp[src][k] = 0，也就是从 src 到 src，不管换乘站是多少，费用都为 0，另外如果 flights 中某个航班 flight 是从 src 站飞往目的地 flight[1]，那么也要初始化 dp[flight[1][0] = flight[2]，也就是要初始化和 src 直接相连的节点。

有了 dp 数组和初始值之后，状态转移方程如下： dp[flight[1]][k] = max(dp[flight[1]][k], dp[flight[0]][k - 1] + flight[2])

其实上述解法就是从 0 个换乘站开始，找到从 src 经过 0 个换乘站到达其余各站的最小花费（也就是和 src 直接相连的点），然后接着就是求从 src 经过 1 个换乘站到达其余各站的最小花费，依次类推，最后求出从 src 经过 k 个换乘站到达其余各站的最小花费。然后 dp[dst][k] 就是我们所要求的。

In [1]:
class Solution(object):
    def findCheapestPrice(self, n, flights, src, dst, k):
        dist = [float('inf')] * n  # dist[v]表示到达v的最小花费
        dist[src] = 0

        for i in range(k + 1):  # 对每条边做 k+1 次松弛操作
            dist_old = [_ for _ in dist]
            for u, v, w in flights:
                dist[v] = min(dist[v], dist_old[u] + w)

        return dist[dst] if dist[dst] != float('inf') else -1

In [ ]:
public class CheapestFlightsWithinKStops {
    public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        // dp[i][k]是经过k个中转站后到达站 i 的最小费用
        int[][] dp = new int[n][K + 1];

        // 循环初始化整个二维数组。
        for(int i = 0; i < n; ++i) Arrays.fill(dp[i], Integer.MAX_VALUE);

        // 利用flights中的信息初始化src可直达的班次
        for(int[] flight : flights) {
            if(flight[0] == src){
                dp[flight[1]][0] = flight[2];
            }
        }

        // 循环初始化数组中dst == src的行
        for(int i = 0; i <= K; i++){
            dp[src][i] = 0;
        }

        //动态规划状态转移方程，开始填表
        //直达的已经初始化了（即k = 0的情况），现在从k = 1 的开始，即只有一个中转站开始
        for(int k = 1; k <= K; k++){
            for(int[] flight : flights){
                //结合题目理解
                if(dp[flight[0]][k - 1] != Integer.MAX_VALUE){
                    dp[flight[1]][k] = Math.min(dp[flight[1]][k], dp[flight[0]][k - 1] + flight[2]);
                }
            }
        }
        return dp[dst][K] == Integer.MAX_VALUE? -1: dp[dst][K];
    }
}

## BFS

In [ ]:
class Solution:
	def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, K: int) -> int:
		# K => bfs
		if flights == [] or src == dst : return 0 # special case
		queue = collections.deque([(src, 0)])
		cost = [float('inf')] * n
		cost[src] = 0
		adjList = collections.defaultdict(list)
		costList = collections.defaultdict(tuple)
		for x, y, c in flights:
			adjList[x].append(y)
			costList[(x, y)] = c
			
		for _ in range(K + 1):
			length = len(queue)
			for i in range(length):
				top, c = queue.popleft()
				for nei in adjList[top]:
					if costList[(top, nei)] + c < cost[nei]:
						cost[nei] = costList[(top, nei)] + c
						queue.append((nei, cost[nei]))
		return cost[dst] if cost[dst] != float('inf') else -1

## DFS

In [ ]:
from collections import defaultdict
import heapq

class Solution:
    def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, K: int) -> int:
        if not flights:
            return -1
        src2dst = defaultdict(list)
        for u, v, w in flights:
            src2dst[u].append((v, w))
        # prices, number of stops, current stop
        heap = [(0, 0, src)]
        res = float('inf')

        while heap:
            price, stop, cur = heapq.heappop(heap)
            if cur == dst:
                res = min(res, price)
                break
            if stop > K:
                continue
            for nxt, cost in src2dst[cur]:
                heapq.heappush(heap, (price + cost, stop + 1, nxt))
            # print(heap)
        return res if res != float('inf') else -1

In [ ]:
class Solution {
   public int findCheapestPrice(int n, int[][] flights, int src, int dst, int K) {
        int[][] g = new int[n][n];
        for (int[] f : flights) {
            g[f[0]][f[1]] = f[2];
        }
        PriorityQueue<int[]> heap = new PriorityQueue<>((a, b) -> a[0] - b[0]);
        heap.add(new int[]{0, src, K + 1});

        while (!heap.isEmpty()) {
            int[] cur = heap.poll();
            int price = cur[0], place = cur[1], remainStops = cur[2];
            if (place == dst) {
                return price;
            }
            if (remainStops > 0) {
                for (int i = 0; i < n; i++) {
                    if (g[place][i] > 0) {
                        heap.offer(new int[]{price + g[place][i], i, remainStops - 1});
                    }
                }
            }
        }
        return -1;
    }
}